In [1]:
pip install bigml --user

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install kaggle --user

Note: you may need to restart the kernel to use updated packages.


In [3]:
import bigml.api
from bigml.api import BigML

In [4]:
import sklearn as sk
import numpy as np
import pandas as pd

In [5]:
from sklearn.model_selection import train_test_split

In [6]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split

In [7]:
from pandas import read_csv

In [8]:
api=BigML ('rodolphevdl','a47cb24c1d3a04762b4b5db6ece5e4c915efa86c',project ='project/5d9d9dd05a2139288900006f')

In [9]:
source = api.create_source('KFP.csv')

In [10]:
source_test = api.create_source('KFP_dataset_test_kaggle.csv')

In [11]:
dataset = api.create_dataset(source)

In [12]:
dataset_test = api.create_dataset(source_test)

In [13]:
train_dataset = api.create_dataset(dataset, {"name": "KFP|train 80% ", "sample_rate": 0.8})
val_dataset = api.create_dataset(dataset, {"name": "KFP|validation 20% ", "sample_rate": 0.8, "out_of_bag": True})

In [14]:
ensemble_args = {"objective_field": "SeriousDlqin2yrs"}
ensemble = api.create_ensemble(train_dataset, ensemble_args)

In [16]:
evaluation = api.create_evaluation(ensemble,val_dataset)

In [17]:
api.ok(evaluation)

True

In [18]:
prediction_args = {"name": "prediction"}
batch_prediction = api.create_batch_prediction(ensemble, val_dataset, {
    "header": True,
    "all_fields": True,
    "probabilities": True})

In [19]:
api.ok(batch_prediction)

True

In [24]:
api.download_dataset(val_dataset, filename='KFPKaggleVal.csv')

'KFPKaggleVal.csv'

In [21]:
api.download_batch_prediction(batch_prediction, filename='KFPKaggleBatch.csv')

'KFPKaggleBatch.csv'

In [22]:
prediction_args = {"name": "Prediction_full_kaggle"}
batch_prediction2 = api.create_batch_prediction(ensemble, dataset_test, {
    "header": True,
    "all_fields": True,
    "probabilities": True})

In [25]:
api.download_batch_prediction(batch_prediction, filename='KFPKaggleBatch2.csv')

'KFPKaggleBatch2.csv'

In [26]:
df = read_csv("KFPKaggleBatch2.csv")
df2 = df[['1 probability']]

In [27]:
df2.index = np.arange(1,len(df2)+1)
df2.index.names = ['Id']

In [31]:
df2.rename(columns = {'1 probability':'Probability'}, inplace=True)

In [32]:
#df2= df2.iloc[[i for i in range(0,101503)]]

In [33]:
df2.to_csv('KFPKaggleBatch2.csv')

In [34]:
import kaggle
submission_file = 'KFPKaggleBatch2.csv'

In [35]:
%env KAGGLE_USERNAME=setsin
%env KAGGLE_KEY=26cbe7b5aa9036281f4eb50023c0b353

env: KAGGLE_USERNAME=setsin
env: KAGGLE_KEY=26cbe7b5aa9036281f4eb50023c0b353


In [36]:
print(submission_file)

KFPKaggleBatch2.csv


In [37]:
!head ./kaggle-submission.csv

Id,Probability
5,0.22824
6,0.01113
7,0.0282
14,0.7185600000000001
19,0.02543
22,0.01784
34,0.01212
35,0.16508
38,0.11731


In [38]:
kaggle.api.competition_submit('KFPKaggleBatch2.csv', "Ensemble", "GiveMeSomeCredit")

100%|██████████| 490k/490k [00:26<00:00, 19.2kB/s] 


Successfully submitted to Give Me Some Credit